In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
rcParams['figure.dpi'] = 100
rcParams['figure.facecolor'] = (1,1,1,1)

In [3]:
import xarray as xr
import numpy as np
import pandas as pd

from tqdm import tqdm

import cartopy.crs as ccrs

from pathlib import Path

In [4]:
import io

In [5]:
import warnings

In [6]:
import dask
from dask.distributed import Client
import dask.bag

In [7]:
def get_dask_client():
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        c = Client('localhost:8786')
    return c

In [36]:
client = get_dask_client()
client

Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 32 Cores: 32 Memory: 192.00 GB


In [11]:
from datetime import datetime, timezone

In [12]:
import PIL.Image

In [13]:
import sys 
sys.path.append('/ships19/cloud/scratch/cphillips/utility')
from make_gif import make_video, render_fast, play_video
sys.path.append('/data/mstengel/ISCCP/code_save/')
import rgb

In [37]:
# Upload
# Upload
# Upload
client.upload_file(rgb.__file__)

In [15]:
OUT_DIR = Path('../final/2020/').resolve().absolute()

In [16]:
VID = Path('videos')
VID.mkdir(exist_ok=True)

In [38]:
dirs = sorted(OUT_DIR.glob('202006/*/*'))
len(dirs)

1440

In [39]:
def get_im(args, fillna=False):
    refl_00_65um_f, refl_00_86um_f, refl_01_60um_f, temp_11_00um_f, solzen_f = args
    
    datasets = {}
    
    datasets['refl_00_65um'] = xr.open_dataset(refl_00_65um_f)
    datasets['refl_00_86um'] = xr.open_dataset(refl_00_86um_f)
    datasets['refl_01_60um'] = xr.open_dataset(refl_01_60um_f)
    datasets['temp_11_00um'] = xr.open_dataset(temp_11_00um_f)
    solzen = xr.open_dataset(solzen_f)
    
    if not fillna:
        for k, ds in sorted(datasets.items()):
            datasets[k] = ds[k].isel(layer=0, time=0)
        mask = (datasets['refl_00_65um'].notnull()
                & datasets['refl_00_86um'].notnull()
                & datasets['refl_01_60um'].notnull()
                & datasets['temp_11_00um'].notnull())
        for k, ds in sorted(datasets.items()):
            datasets[k] = ds.where(mask)
    else:
        for k, ds in sorted(datasets.items()):
            datasets[k] = ds[k].isel(layer=0, time=0).fillna(ds[k].isel(layer=1,time=0).fillna(ds[k].isel(layer=2,time=0)))

    im = rgb.calc_rgb_with016(datasets['refl_00_65um'].values.squeeze(),
                     datasets['refl_00_86um'].values.squeeze(),
                     datasets['refl_01_60um'].values.squeeze(),
                     datasets['temp_11_00um'].values.squeeze(),
                     solzen['solar_zenith_angle'].values.squeeze()).astype(np.uint8)
    ds.close()
    im = PIL.Image.fromarray(im[1440:2160,800:800+1280,:])
    #return im
    fp = io.BytesIO()
    #im = im.resize((2048,1024))
    im.save(fp, format='jpeg')
    fp.seek(0)
    return fp.read()

In [40]:
def get_im_layer0(args):
    return get_im(args)
def get_im_fillna(args):
    return get_im(args, fillna=True)

In [41]:
from collections import defaultdict

In [42]:
days = defaultdict(list)
for d in dirs:
    date = datetime.strptime(d.name, '%Y%m%dT%H%M').replace(tzinfo=timezone.utc)
    try:
        refl_00_65um = next(d.glob('*_refl_00_65um_2*'))
        refl_00_86um = next(d.glob('*_refl_00_86um_2*'))
        refl_01_60um = next(d.glob('*_refl_01_60um_2*'))
        temp_11_00um = next(d.glob('*_temp_11_00um_2*'))
        solar_zenith_angle = next(d.glob('*_solar_zenith_angle_2*'))
        days[date.date()].append((refl_00_65um, refl_00_86um, refl_01_60um, temp_11_00um,solar_zenith_angle))
    except StopIteration:
        print(f'Missing files {date}')

Missing files 2020-06-01 16:30:00+00:00
Missing files 2020-06-01 21:00:00+00:00
Missing files 2020-06-02 03:30:00+00:00
Missing files 2020-06-02 04:00:00+00:00
Missing files 2020-06-02 04:30:00+00:00
Missing files 2020-06-02 05:00:00+00:00
Missing files 2020-06-02 05:30:00+00:00
Missing files 2020-06-02 06:00:00+00:00
Missing files 2020-06-02 06:30:00+00:00
Missing files 2020-06-02 07:00:00+00:00
Missing files 2020-06-02 07:30:00+00:00
Missing files 2020-06-02 08:00:00+00:00
Missing files 2020-06-02 08:30:00+00:00
Missing files 2020-06-02 09:00:00+00:00
Missing files 2020-06-02 09:30:00+00:00
Missing files 2020-06-02 10:00:00+00:00
Missing files 2020-06-02 10:30:00+00:00
Missing files 2020-06-02 11:00:00+00:00
Missing files 2020-06-02 11:30:00+00:00
Missing files 2020-06-02 12:00:00+00:00
Missing files 2020-06-02 12:30:00+00:00
Missing files 2020-06-02 13:00:00+00:00
Missing files 2020-06-02 13:30:00+00:00
Missing files 2020-06-02 14:00:00+00:00
Missing files 2020-06-02 14:30:00+00:00


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:42800 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:42812 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:42828 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:42880 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:42894 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:42912 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:42984 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:42988 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing danglin

In [28]:
%%time
for day, day_tasks in sorted(days.items()):
    print(day)
    tasks_bag = dask.bag.from_sequence(list(day_tasks)[:48], partition_size=1)
    video = VID / day.strftime('true_color_l1g_layer0_%Y%m%d.mp4')
    if not video.exists():
        try:
            with make_video(video, framerate=8, bitrate="80000k") as vid:
                print('Rendering frames')
                frames = tasks_bag.map(get_im_fillna).compute()
                print('Done')
                for frame in frames:
                    vid.write(frame)
        except OSError:
            print('Error', day)

2020-05-01
Rendering frames
Done
2020-05-02
Rendering frames
Done
2020-05-03
Rendering frames
Done
2020-05-04
Rendering frames
Done
2020-05-05
Rendering frames
Done
2020-05-06
Rendering frames
Done
2020-05-07
Rendering frames
Done
2020-05-08
Rendering frames
Done
2020-05-09
Rendering frames
Done
2020-05-10
Rendering frames
Done
2020-05-11
Rendering frames
Done
2020-05-12
Rendering frames
Done
2020-05-13
Rendering frames
Done
2020-05-14
Rendering frames
Done
2020-05-15
Rendering frames
Done
2020-05-16
Rendering frames
Done
2020-05-17
Rendering frames
Done
2020-05-18
Rendering frames
Done
2020-05-19
Rendering frames
Done
2020-05-20
Rendering frames
Done
2020-05-21
Rendering frames
Done
2020-05-22
Rendering frames
Done
2020-05-23
Rendering frames
Done
2020-05-24
Rendering frames
Done
2020-05-25
Rendering frames
Done
2020-05-26
Rendering frames
Done
2020-05-27
Rendering frames
Done
2020-05-28
Rendering frames
Done
2020-05-29
Rendering frames
Done
2020-05-30
Rendering frames
Done
2020-05-31

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:41616 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:41640 remote=tcp://localhost:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://[::1]:41688 remote=tcp://localhost:8786>


In [71]:
Path('/ships19/cloud/scratch/cphillips/isccp_l1g/final/2020/202001/20200118/20200118T2130/ISCCP-NG_L1g_demo_A1_v1_res_0_10deg__temp_11_00um_20200118T2130.nc').unlink()

In [138]:
video.stat().st_size/1e6

5.648029

In [139]:
play_video(video)

In [46]:
video

PosixPath('videos/true_color_l1g_layer0_20200101.mp4')